# Simple Lya P1D forecasts using minimizer and MCMC

In [1]:
%matplotlib inline
import os
import numpy as np
from cobaya.yaml import yaml_load_file
from cobaya.run import run

Load pre-run chains for Lya BAO and gal BAO

In [2]:
info = yaml_load_file('inifiles/test.yaml')
info['force']=True
info['debug']=False
_, sampler = run(info)

[output_mpi] Output to be read-from/written-into folder 'minimum', with prefix 'test'
[camb] Importing *auto-installed* CAMB (but defaulting to *global*).
[camb] Initialized!
[minimize] Initializing
[minimize] Starting from random initial point:
[minimize] {'ns': 0.9636834800892071, 'logA': 2.9832137507029217}
[minimize] Starting minimization.
[minimize] Finished successfully!
[minimize] -log(posterior) minimized to -0.776503
[minimize] Parameter values at minimum:
   weight  minuslogpost       ns      logA            As     ombh2     omch2  minuslogprior  minuslogprior__0      chi2  chi2__lyalike.lya.Tester
0     1.0     -0.776503  0.96446  3.029459  2.068604e-09  0.021996  0.120109      -0.776529         -0.776529  0.000052                  0.000052


In [3]:
def get_covariance(sampler):
    p = sampler.products()
    r = p['result_object']
    try:
        # method: scipy
        inv_hess = r.hess_inv.todense()
    except:
        # method: bobyqa
        inv_hess = np.linalg.inv(r.hessian)
    cov=p['M'].T.dot(inv_hess).dot(p['M'])
    return cov

In [5]:
cov=get_covariance(sampler)
min_err=np.sqrt(cov.diagonal())
min_ns=sampler.products()['minimum']['ns']
min_logA=sampler.products()['minimum']['logA']
print('ns = {:.5e} +/- {:.5e}'.format(min_ns,min_err[0]))
print('logA = {:.5e} +/- {:.5e}'.format(min_logA,min_err[1]))

ns = 9.64460e-01 +/- 2.94586e-03
logA = 3.02946e+00 +/- 9.82473e-03


### Redo with MCMC chains

In [10]:
info_mcmc = yaml_load_file('inifiles/test.yaml')
info_mcmc['sampler']={'mcmc':{'max_tries':1000}}
info_mcmc['force']=True
info_mcmc['debug']=False
updated_info_mcmc, sampler_mcmc = run(info_mcmc)

[output_mpi] Output to be read-from/written-into folder 'minimum', with prefix 'test'
[camb] Importing *auto-installed* CAMB (but defaulting to *global*).
[camb] Initialized!
[mcmc] Getting initial point... (this may take a few seconds)
[model] Measuring speeds... (this may take a few seconds)
[model] Setting measured speeds (per sec): {lyalike.lya.Tester: 29100.0, camb.transfers: 1.37, camb: 1430.0, lyalike.lya.Lya: 1450.0}
[mcmc] Covariance matrix not present. We will start learning the covariance of the proposal earlier: R-1 = 30 (would be 2 if all params loaded).
[mcmc] Initial point: ns:0.9704006, logA:2.845645
[mcmc] Sampling!
[mcmc] Progress @ 2021-03-15 05:23:17 : 1 steps taken, and 0 accepted.
[mcmc] Learn + convergence test @ 80 samples accepted.
[mcmc]  - Acceptance rate: 0.045
[mcmc]  - Convergence of means: R-1 = 0.191041 after 80 accepted steps
[mcmc]  - Updated covariance matrix of proposal pdf.
[mcmc] Learn + convergence test @ 160 samples accepted.
[mcmc]  - Acceptance

In [11]:
# Export the results to GetDist
from getdist.mcsamples import MCSamplesFromCobaya

gd_sample = MCSamplesFromCobaya(updated_info_mcmc, sampler_mcmc.products()["sample"])
# Analyze and plot
mcmc_mean = gd_sample.getMeans()[:2]
mcmc_cov = gd_sample.getCovMat().matrix[:2, :2]
mcmc_err = np.sqrt(mcmc_cov.diagonal())
print('mcmc ns = {:.5e} +/- {:.5e}'.format(mcmc_mean[0],mcmc_err[0]))
print('mcmc logA = {:.5e} +/- {:.5e}'.format(mcmc_mean[1],mcmc_err[1]))

[root] *WARNING* outlier fraction 0.009935897435897435 
mcmc ns = 9.64356e-01 +/- 3.11527e-03
mcmc logA = 3.02911e+00 +/- 9.46103e-03


In [12]:
print('minimizer ns = {:.5e} +/- {:.5e}'.format(min_ns,min_err[0]))
print('minimizer logA = {:.5e} +/- {:.5e}'.format(min_logA,min_err[1]))

minimizer ns = 9.64460e-01 +/- 2.94586e-03
minimizer logA = 3.02946e+00 +/- 9.82473e-03
